In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import (BartTokenizer, BartForConditionalGeneration,
                          T5Tokenizer, T5ForConditionalGeneration,
                          PegasusTokenizer, PegasusForConditionalGeneration,
                          BertTokenizer, BertForSequenceClassification,
                          Trainer, TrainingArguments)
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu
import xgboost as xgb
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

# Load SpaCy's English model
nlp = spacy.load('en_core_web_sm')

# Load the WikiText dataset
dataset = load_dataset('wikitext', 'wikitext-103-raw-v1')

# Accessing the dataset directly and converting to a list of texts
texts = dataset['test']['text'][:100]  # Load a small subset for demonstration

# Split dataset into train, validation, and test sets
train_texts, temp_texts = train_test_split(texts, test_size=0.2, random_state=42)
val_texts, test_texts = train_test_split(temp_texts, test_size=0.5, random_state=42)

# Tokenization and preprocessing functions
def preprocess_texts(texts, tokenizer, max_length=512):
    inputs = tokenizer(texts, max_length=max_length, padding=True, truncation=True, return_tensors="pt")
    return inputs

# Fine-tuning Summarization Models
def fine_tune_model(model, tokenizer, train_texts, val_texts, epochs=3):
    # Prepare the inputs for the model
    train_inputs = preprocess_texts(train_texts, tokenizer)
    val_inputs = preprocess_texts(val_texts, tokenizer)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=epochs,
        weight_decay=0.01
    )

    # Trainer object
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_inputs,
        eval_dataset=val_inputs
    )

    # Fine-tune the model
    trainer.train()

# Fine-tune BART
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
fine_tune_model(bart_model, bart_tokenizer, train_texts, val_texts)

# Fine-tune T5
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
fine_tune_model(t5_model, t5_tokenizer, train_texts, val_texts)

# Fine-tune Pegasus
pegasus_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
pegasus_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
fine_tune_model(pegasus_model, pegasus_tokenizer, train_texts, val_texts)

# Classification fine-tuning
def fine_tune_bert_model(train_texts, train_labels, val_texts, val_labels, epochs=3):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    train_inputs = preprocess_texts(train_texts, tokenizer)
    val_inputs = preprocess_texts(val_texts, tokenizer)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=epochs,
        weight_decay=0.01
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_inputs,
        eval_dataset=val_inputs
    )

    # Fine-tune the model
    trainer.train()

# Dummy labels (you would replace this with actual labels from your dataset)
train_labels = np.random.randint(0, 2, size=len(train_texts))
val_labels = np.random.randint(0, 2, size=len(val_texts))

# Fine-tune BERT for classification
fine_tune_bert_model(train_texts, train_labels, val_texts, val_labels)

# Summarization functions (after fine-tuning)
def summarize_with_model(model, tokenizer, texts, max_length=150):
    summaries = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        summary_ids = model.generate(inputs.input_ids, num_beams=4, max_length=max_length, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

# Summarize fine-tuned BART
fine_tuned_bart_summaries = summarize_with_model(bart_model, bart_tokenizer, train_texts)

# Summarize fine-tuned T5
fine_tuned_t5_summaries = summarize_with_model(t5_model, t5_tokenizer, train_texts)

# Summarize fine-tuned Pegasus
fine_tuned_pegasus_summaries = summarize_with_model(pegasus_model, pegasus_tokenizer, train_texts)

# Classification functions (after fine-tuning)
def classify_with_bert(texts):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1).numpy()

    return predictions

# Evaluate fine-tuned models on classification and summarization tasks
def evaluate_summarization(original_texts, summaries):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bleu_scores = []
    for orig, sum_text in zip(original_texts, summaries):
        rouge_scores = scorer.score(orig, sum_text)
        bleu_scores.append(sentence_bleu([orig.split()], sum_text.split()))
    return rouge_scores, bleu_scores

def evaluate_classification(true_labels, predictions):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    return accuracy, precision, recall

# Evaluate summarization
rouge_bart, bleu_bart = evaluate_summarization(train_texts, fine_tuned_bart_summaries)
rouge_t5, bleu_t5 = evaluate_summarization(train_texts, fine_tuned_t5_summaries)
rouge_pegasus, bleu_pegasus = evaluate_summarization(train_texts, fine_tuned_pegasus_summaries)

# Evaluate classification
bert_predictions = classify_with_bert(fine_tuned_bart_summaries)
accuracy_bert, precision_bert, recall_bert = evaluate_classification(train_labels, bert_predictions)

# Print results
print("ROUGE Scores (Fine-tuned BART):", rouge_bart)
print("BLEU Scores (Fine-tuned BART):", bleu_bart)
print("Accuracy (Fine-tuned BERT):", accuracy_bert)
print("Precision (Fine-tuned BERT):", precision_bert)
print("Recall (Fine-tuned BERT):", recall_bert)
